<a href="https://colab.research.google.com/github/fuyu-quant/Data_Science/blob/main/Tabel_Data/AutoML/H2o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# H2o
* 参考記事  
https://qiita.com/DS27/items/daf4aee39741cc531c27

In [1]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.5 MB 42 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.3-py2.py3-none-any.whl size=177541401 sha256=4f8b30d04c3be2d050dcf98577d96e6bd359922198139c25565bd8e8227bf528
  Stored in directory: /root/.cache/pip/wheels/16/f8/f4/69e1ff6a0d1cb61bdbc0d9888ee8437a1acf1eb1c6ffb8be20
Successfully built h2o


In [3]:
import h2o
from h2o.automl import H2OAutoML

import sys, os, os.path
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [18]:
import h2o
from h2o.automl import H2OAutoML

import sys, os, os.path
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

train = pd.merge(x_train, y_train, left_index=True, right_index=True)
test = pd.merge(x_test, y_test, left_index=True, right_index=True)

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

# 説明変数/目的変数のカラムを指定
kw = h2o.H2OFrame(list(h2o_train.columns))

feature_cols = kw.columns[:-1]
target_cols = "MEDV"

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 mins 44 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_u1gz3u
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.00 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
# max_runtime_secsは最大実行時間を指定
aml = H2OAutoML(
    max_runtime_secs=30,  # 30 sec
    max_models=None,  # no limit
    stopping_metric ='rmse',
    sort_metric ='rmse',
    seed = RANDOM_STATE,
    )

aml.train(
    y=target_cols,
    training_frame=h2o_train,
    )

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_6_20221219_152300_model_2


Model Summary: 
    number_of_trees
--  -----------------
    41

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.482577033744934
RMSE: 0.694677647362382
MAE: 0.4062027282053881
RMSLE: 0.03394828900945341
Mean Residual Deviance: 0.482577033744934

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 9.940858797884673
RMSE: 3.1529127482194417
MAE: 2.158922836332038
RMSLE: 0.148158479958285
Mean Residual Deviance: 9.940858797884673

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
mae                     2.15854   0.232888   2.22418       2.09812       2.52808       1.93795       2.00437
mean_residual_deviance  9.93347   3.0087     13.1352       8.52507       13.2083       7.85051       6.94828
mse                     9.93347   3.0087     13.1352       8.52507       13.2083       7.85051       6.94828
r2                      0.866248  0.0514127  0.81559       0.881816      0.808623      0.904062      0.921147
residual_deviance       9.93347   3.0087     13.1352       8.52507       13.2083       7.85051       6.94828
rmse                    3.12324   0.472846   3.62425       2.91977       3.63432       2.80188       2.63596
rmsle                   0.146932  0.0206401  0.150835      0.151315      0.173325      0.143227      0.11596

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2022-12-19 15:23:27  2.698 sec   0                  23.0745          21.3418         532.435
    2022-12-19 15:23:27  2.735 sec   5                  5.17326          4.0356          26.7626
    2022-12-19 15:23:27  2.752 sec   10                 2.3016           1.50832         5.29737
    2022-12-19 15:23:27  2.762 sec   15                 1.69293          1.03863         2.86601
    2022-12-19 15:23:27  2.776 sec   20                 1.32986          0.823563        1.76852
    2022-12-19 15:23:27  2.791 sec   25                 1.09074          0.68704         1.1897
    2022-12-19 15:23:27  2.813 sec   30                 0.936167         0.578952        0.876409
    2022-12-19 15:23:27  2.835 sec   35                 0.817721         0.482801        0.668667
    2022-12-19 15:23:27  2.855 sec   40                 0.713024         0.417937        0.508404
    2022-12-19 15:23:27  2.873 sec   41                 0.694678         0.406203        0.482577

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
LSTAT       23859.9                1                    0.447919
RM          17953.6                0.752459             0.337041
PTRATIO     3837.99                0.160855             0.0720501
DIS         2272.6                 0.0952475            0.0426632
NOX         1987.43                0.0832957            0.0373097
CRIM        1365.34                0.0572232            0.0256313
B           626.181                0.0262441            0.0117552
AGE         593.366                0.0248688            0.0111392
TAX         430.851                0.0180575            0.00808831
INDUS       215.3                  0.0090235            0.00404179
RAD         110.783                0.00464308           0.00207972
ZN          10.4559                0.000438223          0.000196288
CHAS        4.56509                0.000191329          8.57e-05

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [20]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
XGBoost_grid_1_AutoML_6_20221219_152300_model_2,3.15291,9.94086,2.15892,0.148158,9.94086
StackedEnsemble_AllModels_2_AutoML_6_20221219_152300,3.15477,9.95256,2.12372,0.145673,9.95256
GBM_3_AutoML_6_20221219_152300,3.15566,9.95818,2.13075,0.149529,9.95818
StackedEnsemble_BestOfFamily_2_AutoML_6_20221219_152300,3.16526,10.0189,2.14073,0.145203,10.0189
StackedEnsemble_BestOfFamily_3_AutoML_6_20221219_152300,3.16784,10.0352,2.14854,0.146725,10.0352
GBM_2_AutoML_6_20221219_152300,3.18732,10.159,2.16197,0.146695,10.159
StackedEnsemble_AllModels_1_AutoML_6_20221219_152300,3.19589,10.2137,2.13671,0.146475,10.2137
GBM_4_AutoML_6_20221219_152300,3.24011,10.4983,2.15958,0.152872,10.4983
XGBoost_2_AutoML_6_20221219_152300,3.32924,11.0838,2.39281,0.157668,11.0838
GBM_5_AutoML_6_20221219_152300,3.43255,11.7824,2.26486,0.15716,11.7824


In [21]:
aml.leader

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_6_20221219_152300_model_2


Model Summary: 
    number_of_trees
--  -----------------
    41

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.482577033744934
RMSE: 0.694677647362382
MAE: 0.4062027282053881
RMSLE: 0.03394828900945341
Mean Residual Deviance: 0.482577033744934

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 9.940858797884673
RMSE: 3.1529127482194417
MAE: 2.158922836332038
RMSLE: 0.148158479958285
Mean Residual Deviance: 9.940858797884673

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
mae                     2.15854   0.232888   2.22418       2.09812       2.52808       1.93795       2.00437
mean_residual_deviance  9.93347   3.0087     13.1352       8.52507       13.2083       7.85051       6.94828
mse                     9.93347   3.0087     13.1352       8.52507       13.2083       7.85051       6.94828
r2                      0.866248  0.0514127  0.81559       0.881816      0.808623      0.904062      0.921147
residual_deviance       9.93347   3.0087     13.1352       8.52507       13.2083       7.85051       6.94828
rmse                    3.12324   0.472846   3.62425       2.91977       3.63432       2.80188       2.63596
rmsle                   0.146932  0.0206401  0.150835      0.151315      0.173325      0.143227      0.11596

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2022-12-19 15:23:27  2.698 sec   0                  23.0745          21.3418         532.435
    2022-12-19 15:23:27  2.735 sec   5                  5.17326          4.0356          26.7626
    2022-12-19 15:23:27  2.752 sec   10                 2.3016           1.50832         5.29737
    2022-12-19 15:23:27  2.762 sec   15                 1.69293          1.03863         2.86601
    2022-12-19 15:23:27  2.776 sec   20                 1.32986          0.823563        1.76852
    2022-12-19 15:23:27  2.791 sec   25                 1.09074          0.68704         1.1897
    2022-12-19 15:23:27  2.813 sec   30                 0.936167         0.578952        0.876409
    2022-12-19 15:23:27  2.835 sec   35                 0.817721         0.482801        0.668667
    2022-12-19 15:23:27  2.855 sec   40                 0.713024         0.417937        0.508404
    2022-12-19 15:23:27  2.873 sec   41                 0.694678         0.406203        0.482577

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
LSTAT       23859.9                1                    0.447919
RM          17953.6                0.752459             0.337041
PTRATIO     3837.99                0.160855             0.0720501
DIS         2272.6                 0.0952475            0.0426632
NOX         1987.43                0.0832957            0.0373097
CRIM        1365.34                0.0572232            0.0256313
B           626.181                0.0262441            0.0117552
AGE         593.366                0.0248688            0.0111392
TAX         430.851                0.0180575            0.00808831
INDUS       215.3                  0.0090235            0.00404179
RAD         110.783                0.00464308           0.00207972
ZN          10.4559                0.000438223          0.000196288
CHAS        4.56509                0.000191329          8.57e-05

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [22]:
y_pred = aml.leader.predict(h2o_test)


xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


## データセットの用意

In [10]:
iris_dataset = load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris_dataset['data'], iris_dataset['target'], test_size=0.25,  random_state=0)

train = pd.DataFrame(x_train, columns=iris_dataset.feature_names)
train['target'] = y_train

test = pd.DataFrame(x_test, columns=iris_dataset.feature_names)
test['target'] = y_test

## H2oの設定

In [6]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpxvyx0c3s
  JVM stdout: /tmp/tmpxvyx0c3s/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpxvyx0c3s/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_u1gz3u
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [11]:
#train = pd.merge(x_train, y_train, left_index=True, right_index=True)
#test = pd.merge(x_test, y_test, left_index=True, right_index=True)

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
h2o_train.columns

In [12]:
kw = h2o.H2OFrame(list(h2o_train.columns))

feature_cols = kw.columns[:-1]
target_cols = "target"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


## H2oの学習

In [17]:
H2o_model = H2OAutoML(max_runtime_secs=1000,  # 30 sec
                max_models=None,  # no limit
                #stopping_metric ='rmse',
                #stopping_metric = 'AUTO', #This defaults to logloss for classification, deviance for regression
                #stopping_metric = 'deviance',
                stopping_metric = 'logloss',
                #stopping_metric = 'auc',
                #sort_metric ='logloss',
                seed = 3655
                )

H2o_model.train(y=target_cols,
          training_frame=h2o_train,
          )

AutoML progress: |
15:21:54.755: _stopping_metric param, Stopping metric cannot be logloss for regression.


15:21:55.113: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.113: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 112.0.
15:21:55.115: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.122: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.127: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.128: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.130: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.132: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.135: _stopping_metric param, Stopping metric cannot be logloss for regression.
15:21:55.141: _stopping_metric par

OSError: ignored